In [114]:
import random
import numpy as np
import math
import time
import os

In [109]:
def euclidean_distance(a, b):
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

In [122]:
def genetic_backpack(max_capacity, w, c):
    N = len(w)
    n_chromo = 4
    dna = np.zeros(N*n_chromo).reshape(n_chromo, N)
    k = 2
    # генерируем хромосомы
    for i in range(n_chromo):
        for j in range(N):
            dna[i][j] = random.randint(0,1)
    #print('Популяция:\n', dna)
    def remove_random_item(chromo):
        one_positions = []
        for i in range(len(chromo)):
            if chromo[i]:
                one_positions.append(i)
        chromo[one_positions[random.randint(0, len(one_positions) - 1)]] = 0
        
    # Считаем вес и стоимость
    def fitness_function(chromo, w, c):
        temp_w, temp_p = 0, 0
        for j in range(N):
            if chromo[j]:
                temp_w += w[j]
                temp_p += c[j]
        return temp_p, temp_w
    total_chromo = []
    for i in range(n_chromo):
        temp_p, temp_w = fitness_function(dna[i], w, c)
        
        while temp_w > max_capacity:
            remove_random_item(dna[i])
            temp_p, temp_w = fitness_function(dna[i], w, c)
        total_chromo.append((i, temp_p, temp_w))
    #print('Популяция после подсчета фит. функции:\n', dna)  
    
    # Сортируем
    total_chromo = sorted(total_chromo, key=lambda x: x[1], reverse=True)
    print(total_chromo)
    
    # Находим распределение вероятностей
    fitness_sum = 0
    for x in total_chromo:
        fitness_sum += x[1]
    distribution = []
    for x in total_chromo:
        distribution.append(x[1] / fitness_sum)
    #distribution.sort(reverse=True)
    
    # выбираем для селекции
    chromos_selected = random.choices(dna, weights=distribution, k=k)
    # чтобы не брать одинаковые
    while len(np.unique(chromos_selected, axis=0)) < k:
        chromos_selected = random.choices(dna, weights=distribution, k=k)
    #print('Выбрали для селекции:\n',chromos_selected)
    
    # делаем селекцию
    crossover_point = random.randint(1, N-1)
    tmp = chromos_selected[1][:crossover_point].copy()
    chromos_selected[1][:crossover_point], chromos_selected[0][:crossover_point] = chromos_selected[0][:crossover_point], tmp
    #print('После селекции:\n', chromos_selected)
    
    # мутация
    for j in range(len(chromos_selected)):
        for i in range(len(chromos_selected[j])):
            if random.randint(1, 100) == 69:
                chromos_selected[j][i] = (chromos_selected[j][i] + 1) % 2
    #print('После мутации:\n', chromos_selected)
    
    # считаем ф-ф полученных хромо, выравниваем веса
    fitness_results_selected = []
    for i in range(len(chromos_selected)):
        temp_p, temp_w = fitness_function(chromos_selected[i], w, c)
        while temp_w > max_capacity:
            remove_random_item(chromos_selected[i])
            temp_p, temp_w = fitness_function(chromos_selected[i], w, c)
        fitness_results_selected.append((i, temp_p, temp_w))

    fitness_results_selected.sort(key=lambda x: x[1], reverse=True)

    if fitness_results_selected[0][1] > total_chromo[0][1]:
        return chromos_selected[fitness_results_selected[0][0]], fitness_results_selected[0][1], fitness_results_selected[0][2]
    else:
        return dna[total_chromo[0][0]], total_chromo[0][1], total_chromo[0][2]
    return

In [108]:
W = [23, 31, 29, 44, 53, 38, 63, 85, 89, 82]
C = [92, 57, 49, 68, 60, 43, 67, 84, 87, 72]
#items, cap, profit = genetic_backpack(10, 165, W, C)
genetic_backpack(10, 165, W, C)

Популяция:
 [[0. 0. 0. 1. 1. 0. 1. 1. 0. 0.]
 [0. 1. 0. 0. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 0. 1. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 0. 0. 1. 0. 1.]]
Популяция после подсчета фит. функции:
 [[0. 0. 0. 1. 1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]]
[(2, 197, 157), (0, 195, 160), (3, 128, 97), (1, 110, 101)]
Выбрали для селекции:
 [array([0., 1., 0., 1., 0., 0., 0., 0., 0., 1.]), array([0., 0., 0., 0., 0., 1., 1., 0., 0., 0.])]
После селекции:
 [array([0., 0., 0., 1., 0., 0., 0., 0., 0., 1.]), array([0., 1., 0., 0., 0., 1., 1., 0., 0., 0.])]
После мутации:
 [array([0., 0., 0., 1., 0., 0., 0., 0., 0., 1.]), array([0., 1., 0., 0., 0., 1., 1., 0., 0., 0.])]


(array([0., 0., 0., 1., 0., 0., 0., 0., 0., 1.]), 197, 157)

In [124]:
p_c = []
p_p = []
p_s = []
p_w = []
for i in range(1, 8):
    p_c.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_c.txt", "r"))
    p_p.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_p.txt", "r"))
    p_s.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_s.txt", "r"))
    p_w.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_w.txt", "r"))
for test_num in range(7):
    print(f"BENCHMARK p0{test_num+1}")
    w = int(p_c[test_num].read())
    items_c = []
    for line in p_p[test_num]:
        items_c.append(int(line))
        
    items_w = []
    for line in p_w[test_num]:
         items_w.append(int(line))

    items_true = []
    w_true = 0
    c_true = 0
    for line in p_s[test_num]:
        items_true.append(int(line))
    for j in range(len(items_true)):
        if items_true[j] == 1:         
            w_true += items_w[j]
            c_true += items_c[j]
                
    answers = []
    timeExecuted = 0
    for i in range(100):
        start = time.time()
        answers.append(genetic_backpack(w, items_w, items_c))
        timeExecuted += time.time() - start
    print(f"TIME: {timeExecuted/100}")
    bestResult = max(list(map(lambda x: x[1], answers)))
    print(f'COST: {bestResult}')
    for i in answers:
        if i[1] == bestResult:
            print(f'ITEMS: {i[0]}')
            break

BENCHMARK p01
[(3, 244, 152), (1, 239, 165), (0, 225, 137), (2, 184, 154)]
[(1, 260, 136), (0, 160, 122), (3, 127, 123), (2, 125, 75)]
[(2, 213, 134), (3, 174, 104), (0, 147, 142), (1, 128, 97)]
[(2, 184, 154), (0, 136, 118), (1, 127, 116), (3, 106, 60)]
[(2, 219, 146), (3, 209, 151), (0, 155, 133), (1, 147, 142)]
[(0, 251, 153), (3, 239, 165), (1, 216, 117), (2, 147, 142)]
[(1, 228, 141), (0, 222, 150), (2, 178, 142), (3, 92, 67)]
[(2, 187, 158), (3, 139, 145), (0, 129, 113), (1, 0, 0)]
[(2, 179, 156), (0, 147, 142), (1, 133, 114), (3, 124, 94)]
[(3, 227, 130), (1, 197, 157), (0, 185, 128), (2, 154, 152)]
[(1, 269, 149), (3, 201, 158), (0, 179, 112), (2, 154, 152)]
[(0, 277, 151), (1, 201, 105), (3, 133, 114), (2, 128, 97)]
[(2, 224, 158), (0, 154, 152), (3, 111, 82), (1, 87, 89)]
[(0, 236, 143), (2, 221, 136), (1, 151, 148), (3, 117, 84)]
[(0, 227, 130), (1, 197, 157), (2, 149, 98), (3, 136, 118)]
[(0, 219, 139), (3, 213, 134), (2, 168, 113), (1, 149, 98)]
[(0, 251, 153), (1, 236, 14

[(3, 1324, 693), (0, 1323, 688), (2, 1314, 685), (1, 1056, 544)]
[(0, 1237, 646), (3, 1221, 637), (1, 1117, 588), (2, 730, 375)]
[(0, 1417, 740), (1, 1183, 620), (3, 746, 390), (2, 664, 345)]
[(0, 1366, 714), (3, 1329, 690), (2, 1328, 694), (1, 1196, 620)]
[(1, 1337, 697), (2, 1305, 672), (3, 1290, 675), (0, 1261, 650)]
[(0, 1443, 750), (2, 1400, 727), (1, 1376, 708), (3, 1262, 656)]
[(3, 1386, 715), (1, 1359, 710), (0, 1284, 668), (2, 653, 346)]
[(0, 1414, 738), (3, 1349, 703), (1, 1046, 550), (2, 747, 390)]
[(2, 1356, 700), (1, 1315, 684), (3, 1315, 684), (0, 1001, 522)]
[(1, 1397, 733), (2, 1391, 721), (0, 1358, 701), (3, 1120, 580)]
[(1, 1450, 748), (2, 1305, 682), (0, 1245, 646), (3, 901, 471)]
[(0, 1316, 682), (3, 1284, 667), (2, 1090, 569), (1, 1015, 527)]
[(3, 1441, 749), (0, 1336, 699), (1, 1136, 590), (2, 1096, 576)]
[(1, 1446, 750), (3, 1434, 749), (2, 1314, 683), (0, 1269, 658)]
[(1, 1380, 712), (0, 1360, 704), (3, 1204, 627), (2, 855, 448)]
[(2, 1309, 682), (3, 1308, 683),

In [ ]:
def fitness_function(chromosome, edges_weight):
    ans = 0
    for i in range(len(chromosome)-1):
        ans += edges_weight[chromosome[i]][chromosome[i+1]]
    return ans


def get_ans(edges_weight):
    num_of_edges = len(edges_weight)
    num_of_chromosomes = 1000
    k = num_of_chromosomes // 4
    k = k // 4
    k *= 4
    chromosomes = []
    for i in range(num_of_chromosomes):
        chromosomes.append([])
        for j in range(num_of_edges):
            chromosomes[i].append(j)
        random.shuffle(chromosomes[i])

    fitness_results = []
    for i in range(num_of_chromosomes):
        tmp = fitness_function(chromosomes[i], edges_weight)
        fitness_results.append((i, tmp))
    fitness_results.sort(key=lambda x: x[1])

    random_list = [0 for i in range(num_of_chromosomes)]
    for i in range(k):
        random_list[i] = 1
    random.shuffle(random_list)

    chromosomes_selected = []
    for i in range(num_of_chromosomes):
        if random_list[i] == 1:
            chromosomes_selected.append((i, fitness_results[i]))

    chromosomes_selected.sort(key=lambda x: x[1])
    chromosomes_selected = chromosomes_selected[:len(chromosomes_selected)//2]

    for i in range(0, len(chromosomes_selected), 2):

        crossover_point1 = random.randint(0, num_of_edges - 2)

        crossover_point2 = random.randint(crossover_point1+1, num_of_edges)

        child1 = [-1 for i in range(num_of_edges)]

        child2 = [-1 for i in range(num_of_edges)]

        tmp1 = chromosomes[chromosomes_selected[i+1][0]][crossover_point2:] + chromosomes[chromosomes_selected[i+1][0]][:crossover_point1]
        tmp2 = chromosomes[chromosomes_selected[i][0]][crossover_point2:] + chromosomes[chromosomes_selected[i][0]][:crossover_point1]

        for j in range(crossover_point1, crossover_point2):
            child1[j] = chromosomes[chromosomes_selected[i][0]][j]
            child2[j] = chromosomes[chromosomes_selected[i+1][0]][j]
            if chromosomes[chromosomes_selected[i+1][0]][j] in tmp1:
                tmp1.remove(chromosomes[chromosomes_selected[i+1][0]][j])
            if chromosomes[chromosomes_selected[i][0]][j] in tmp2:
                tmp2.remove(chromosomes[chromosomes_selected[i][0]][j])

        child1[crossover_point2:] = tmp1[:num_of_edges - crossover_point2]
        child1[:crossover_point1] = tmp1[num_of_edges - crossover_point2:]

        child2[crossover_point2:] = tmp2[:num_of_edges - crossover_point2]
        child2[:crossover_point1] = tmp2[num_of_edges - crossover_point2:]

        if random.randint(1, 100) == 42:
            pointer1 = random.randint(0, num_of_edges - 1)
            pointer2 = random.randint(0, num_of_edges - 1)
            child1[pointer1], child1[pointer2] = child1[pointer2], child1[pointer1]

        if random.randint(1, 100) == 42:
            pointer1 = random.randint(0, num_of_edges - 1)
            pointer2 = random.randint(0, num_of_edges - 1)
            child2[pointer1], child2[pointer2] = child2[pointer2], child2[pointer1]

        chromosomes.append(child1)
        chromosomes.append(child2)

    for i in range(num_of_chromosomes, len(chromosomes)):
        tmp = fitness_function(chromosomes[i], edges_weight)
        fitness_results.append((i, tmp))
    fitness_results.sort(key=lambda x: x[1])

    return fitness_results[0][1], chromosomes[fitness_results[0][0]]

In [ ]:
salesman_benchmarks = {'a280': open('benchmarks/a280.tsp', 'r').readlines(),
                           'att48': open('benchmarks/att48.tsp', 'r').readlines()[6:-1],
                           'bays29': open('benchmarks/bays29.tsp', 'r').readlines()[38:-1],
                           'ch150': open('benchmarks/ch150.tsp', 'r').readlines()[6:-1],
                           'fl417': open('benchmarks/fl417.tsp', 'r').readlines()[6:-1],
                           }

    for benchmark in salesman_benchmarks:
        coord = []
        for line in salesman_benchmarks[benchmark]:
            coord.append((float(line.split()[1]), float(line.split()[2])))
        w = []
        for i in range(len(coord)):
            w.append([])
            for j in range(len(coord)):
                w[i].append(euclidean_distance(coord[i], coord[j]))
        answers = []
        timeExecuted = 0
        for i in range(100):
            start = time.time()
            answer = salesman.get_ans(w)
            timeExecuted += time.time() - start
            answers.append(answer)

        print(f"BENCHMARK {benchmark}")
        print(f"TIME {timeExecuted/100}")
        bestResult = min(list(map(lambda x: x[1], answers)))
        print(f'BEST RESULT: {bestResult}')
        for i in answers:
            if i[1] == bestResult:
                print(f'PATH: {i[0]}')
                break